# Tutorial: A multiple chain chromosomes simulation using Open-MiChroM


The first thing is import the openmichrom module, to install openmm and openmichrom follow the instalation guide on (www..rice.edu)

In [1]:
import openmichrom as OM

In this tutorial we are simulate the chromosome 10 and 11 of human genome GM12878.<br>

To build this system we need start from a compacted structure of each chromossome.<br>
For this, we are perform short simulations of each, separatly, and then, save a structure model of each<br>


Details about single chromosome simulation are in single chromosome tutorial (link)<br>

**Chromosome 10 short simulation**

In [2]:
chr10 = OM.OpenMiChroM(name='chr10', temperature=120, timestep=0.01)
chr10.setup(platform="cuda")
chr10.saveFolder('/home/antonio/multiple_chr/')
mychro = chr10.create_springSpiral(type_list='/home/antonio/openmichrom1.0/Tutorial/multiple_chromosome/chr10_beads.txt')
chr10.load(mychro, center=True)

chr10.addFENEBonds(k=30.0)
chr10.addAngles(k=1.0)
chr10.addRepulsiveSoftCore(Ecut=4.0)
chr10.addSphericalConfinement(density=0.1, k=5)

chr10.addTypetoType(mi=3.22, rc = 1.78)
chr10.addLoops(mi=3.22, rc = 1.78, X=-1.612990,
                 filename=['/home/antonio/openmichrom1.0/Tutorial/single_chromosome/chr10_looplist.txt'])
chr10.addIdealChromosome(mi=3.22, rc = 1.78, dcutl=3, dcutupper=500)

print("Perform chr10 simulation...")
for _ in range(300):
    chr10.doBlock(1000, output=False)

chr10.save(mode="ndb")
del chr10

Perform chr10 simulation...
Number of exceptions:
First neighbor= 2711
Second neighbor= 2710
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force (1 neighbor)
adding force  RepulsiveSoftCore 2
adding force  SphericalConfinement 3
Add exclusions for TypetoType force (1 neighbor)
adding force  TypetoType 4
adding force  Loops 5
Add exclusions for IdealChromosome force (1 neighbor)
adding force  IdealChromosome 6
Positions... 
 loaded!
potential energy is 577.192754


**Chromosome 11 short simulation**

In [ ]:
chr11 = OM.OpenMiChroM(name='chr11', temperature=120, timestep=0.01)
chr11.setup(platform="cuda")
chr11.saveFolder('/home/antonio/multiple_chr/')
mychro = chr11.create_springSpiral(type_list='/home/antonio/openmichrom1.0/Tutorial/multiple_chromosome/chr11_beads.txt')
chr11.load(mychro, center=True)

chr11.addFENEBonds(k=30.0)
chr11.addAngles(k=1.0)
chr11.addRepulsiveSoftCore(Ecut=4.0)
chr11.addSphericalConfinement(density=0.1, k=5)

chr11.addTypetoType(mi=3.22, rc = 1.78)
chr11.addLoops(mi=3.22, rc = 1.78, X=-1.612990,
                 filename=['/home/antonio/openmichrom1.0/Tutorial/multiple_chromosome/chr11_looplist.txt'])
chr11.addIdealChromosome(mi=3.22, rc = 1.78, dcutl=3, dcutupper=500)

print("Perform chr11 simulation...")
for _ in range(300):
    chr11.doBlock(1000, output=False)

chr11.save(mode="ndb")
del chr11

Now we have compacted structures of each chromosome

To visualize the ndb files we can use the ndb converter that can be download 
in https://ndb.rice.edu/ndb-format

We can create the multiple chain configuration using the 2 structures

in OpenMichroM initiation there are some variable to setup:

**timestep=0.01** (the time step using for integration, default is 0.01)<br>
**thermostat=0.1** (the Langevin thermostat, defaut value is 10, e.g., 0.1)<br>
**temperature=120** (Set the temperature of your simulation)<br>
**name='mult_chr10_chr11'** (the simulation name)<br>

In [31]:
chain_C10_c11 = OM.OpenMiChroM(name='mult_chr10_chr11', temperature=120, timestep=0.01)

Now you need to setup the plataform that you use, the options are:

**platform="cuda"** (remenber that you need install CUDA in your system)<br>
**GPU="0" (optional)** (if you have more than one GPU device, you can set the what gpu you wnat ["0", "1",...,"n"])<br>
**platform="cpu"**<br>
**platform="opencl"**<br>

In [32]:
chain_C10_c11.setup(platform="cuda", integrator="Langevin")

Set the Folder name where output are save

In [33]:
chain_C10_c11.saveFolder('/home/antonio/multiple_chr/')

To load structures that we have create above, we use the command: **loadNdb** <br>

loadNdb needs a list of files, even you want load just 1 chromossome.

**Note: Its very important that the ndb files have the same order as the loops files order**

In [34]:
dual_chro = chain_C10_c11.loadNdb(['/home/antonio/multiple_chr/chr10_0_block300.ndb',
                                   '/home/antonio/multiple_chr/chr11_0_block300.ndb'])

Chains:  [(0, 2711, 0), (2712, 5414, 0)]


Dual_chro have the full array of positions of boths chromosomes

The variable chains shows the size of each chromosome in tuples.<br>
where (start,end,is_ring), is_ring=0 represents a linear chain and is_ring=1 when is it a ring

You can call any time this variable to check the chains:

In [35]:
print(chain_C10_c11.chains)

[(0, 2711, 0), (2712, 5414, 0)]


For exemple, to check the chromossome 10 positions (X,Y,Z):

In [36]:
print(dual_chro[chain_C10_c11.chains[0][0]:chain_C10_c11.chains[0][1]+1])

[[-2.242  2.145 -3.158]
 [-2.354  2.192 -4.084]
 [-1.598  2.469 -4.622]
 ...
 [-3.037 -0.263  0.345]
 [-3.503 -0.755 -0.339]
 [-2.807 -1.422 -0.063]]


Before load the structures in the context, we are use a distribution function to set the position of each chromossome in a random way

**note: Its important if want replicas in different initial conditions**

In [37]:
dual_chro = chain_C10_c11.setfibposition(dual_chro, dist=(1.5,3.0))

Now We can load the chromossome in the simulation context

In [38]:
chain_C10_c11.load(dual_chro, center=True)

You can save a structure to visualize de initial position

**when you have multiple chains in the context, the save function will save each structure separatly, each structure have a index star from 0** 

in this case, 0 is for chr10 and 1 is for chr11

In [40]:
chain_C10_c11.save(mode='ndb')

Now its time to include the forces of your system

Lets separete forçes in two sets:

**homopolymer Potentials**

In [41]:
chain_C10_c11.addFENEBonds(k=30.0)
chain_C10_c11.addAngles(k=1.0)
chain_C10_c11.addRepulsiveSoftCore(Ecut=4.0)
chain_C10_c11.addSphericalConfinement(density=0.1, k=5)

23.46947171645543

**Chromosome Potentials**

here we include each force separatly

Types to types intaraction is the same for all

In [42]:
chain_C10_c11.addTypetoType(mi=3.22, rc = 1.78)

The Loops interaction we must be carefull, the file name must follow the same order as the loadNdb funciton

In [43]:
chain_C10_c11.addLoops(mi=3.22, rc = 1.78, X=-1.612990,
                      filename=['/home/antonio/openmichrom1.0/Tutorial/multiple_chromosome/chr10_looplist.txt',
                                '/home/antonio/openmichrom1.0/Tutorial/multiple_chromosome/chr11_looplist.txt']
                      )

To Ideal Chromossome potential we have 1 force for each chromossome

For this, we use another function "addPoliIdealChromosome" <br>
This function recives as input the chain that we want apply this potential

**note: the chains can be read from variable "chains", for exemple, In this case we can call it from the object chain_C10_c11.chains**  

**note 2: here we apply the ideal chromossome from d = 3 to 500, this values can reach the chromosome size if you want**

In [44]:
chain_C10_c11.addPoliIdealChromosome(chain=chain_C10_c11.chains[0], mi=3.22, rc = 1.78, dcutl=3, dcutupper=500)
chain_C10_c11.addPoliIdealChromosome(chain=chain_C10_c11.chains[1], mi=3.22, rc = 1.78, dcutl=3, dcutupper=500)

**The simulation setup is complete!**

Now I will create a file .cndb (binary) to save the frames

The file will be saved in output folder defined earlier<br>
In multiple chain simulations each chromossome will be saved separatly, follow the same ideia described above for save function

In [45]:
chain_C10_c11.initStorage('multChain_C10C11', mode='w')

The Open-MiChroM simulation works in blocks of simulations

for exemple, if want to run $1\times10^8$ steps, saving a frame each $10^3$ steps 

block = 10^3
n_block = 10^5


In [46]:
block = 10**2
n_blocks = 10**2

In [48]:
for _ in range(n_blocks):
    chain_C10_c11.doBlock(block, output=True) #makes output=False if won't print details about simularion 
    chain_C10_c11.save()  #save frame on cndb file

Number of exceptions:
First neighbor= 5413
Second neighbor= 5411
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force (1 neighbor)
adding force  RepulsiveSoftCore 2
adding force  SphericalConfinement 3
Add exclusions for TypetoType force (1 neighbor)
adding force  TypetoType 4
adding force  Loops 5
Add exclusions for IdealChromosome_chain_0 force (1 neighbor)
adding force  IdealChromosome_chain_0 6
Add exclusions for IdealChromosome_chain_2712 force (1 neighbor)
adding force  IdealChromosome_chain_2712 7
Positions... 
 loaded!
potential energy is 37.723863
bl=1 pos[1]=[18.1 15.3 8.9] dr=0.42 t=1.0ps kin=2.30 pot=36.88 Rg=26.490 SPS=131 
bl=2 pos[1]=[17.5 15.1 8.5] dr=0.68 t=2.0ps kin=4.28 pot=34.56 Rg=25.963 SPS=358 
bl=3 (i) pos[1]=[16.6 14.4 8.0] dr=0.94 t=3.0ps kin=6.68 pot=31.41 Rg=25.155 SPS=363 
bl=4 pos[1]=[16.6 14.2 7.9] dr=0.39 t=4.0ps kin=2.13 pot=30.72 Rg=25.011 SPS=362 
bl=5 pos[1]=[16.2 13.9 7.6] dr=0.59 t=5.0ps kin=3.51 pot=29.08 

After the simulation finish you must close the files

In [49]:
chain_C10_c11.storage[0].close() #close the cndb file
chain_C10_c11.storage[1].close() #close the cndb file

If you want to save a structure to visualize in vmd, chimera, etc.. You can save a ndb file.<br>

**The ndb interpreter for visualize in vmd/chimera is under construction<br>**

In [52]:
chain_C10_c11.save(mode="ndb")

To visualize the ndb files we can use the ndb converter that can be download 
in https://ndb.rice.edu/ndb-format